In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

### IMPORTANTE: correr el archivo Data Processing.ipynb que se encuentra en la carpeta Data/ antes de leer los .csv

In [29]:
train = pd.read_csv("Data/train_processed.csv")
test = pd.read_csv("Data/test_processed.csv")
sub_sample = pd.read_csv("Data/sample_submission.csv")

print (train.shape, test.shape, sub_sample.shape)

(7613, 22) (3263, 21) (3263, 2)


In [11]:
train.head()

,Unnamed: 0,id,keyword,location,text,target,text_clean,hashtags,mentions,links,...,stop_word_count,punctuation_count,hashtag_count,mention_count,link_count,caps_count,caps_ratio,location_clean,keyword_target,location_clean_target
0,0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,earthquake,no,no,...,6,1,1,1,1,10,0.144928,None,0.688525,0.424398
1,1,4,None,None,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,no,no,no,...,0,1,1,1,1,5,0.131579,None,0.688525,0.424398
2,2,5,None,None,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,no,no,no,...,11,3,1,1,1,2,0.015038,None,0.688525,0.424398
3,3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",wildfires,no,no,...,1,2,1,1,1,1,0.015625,None,0.688525,0.424398
4,4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,Alaska wildfires,no,no,...,7,2,2,1,1,3,0.034483,None,0.688525,0.424398


In [33]:
test.text.values

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

## LightGBM

Como parte del preprocesamiento se escalan los datos aplicando normalizacion y como modelo LightGBM

In [36]:
# columns_to_drop = ["text_clean", "hashtags", "mentions", "links", "location"]
# X_train = train.drop(columns = columns_to_drop + ['target'])
# X_test = test.drop(columns = columns_to_drop)
y_train = train.target

# creo un token con el texto
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")
X_train = embed(train.text.values)
X_test = embed(test.text.values)


In [39]:
print(X_train["outputs"])

tf.Tensor(
[[-0.0593824  -0.01889507 -0.04077131 ... -0.03765527 -0.06941315
  -0.08671615]
 [ 0.03659685  0.05027486  0.02020235 ... -0.004579   -0.0212087
   0.00290395]
 [ 0.00149931  0.0351244   0.03224244 ... -0.05414995  0.07224016
   0.05789763]
 ...
 [-0.00839222 -0.00212517 -0.0617794  ... -0.05840644 -0.01960979
  -0.08983655]
 [-0.0594376   0.07270394 -0.01466312 ... -0.01396658 -0.06471785
  -0.07085625]
 [ 0.00697963 -0.0717947  -0.01469821 ...  0.00945404  0.02885959
   0.04562907]], shape=(7613, 512), dtype=float32)


In [40]:
print(X_test["outputs"])

tf.Tensor(
[[-0.06240411  0.06200171 -0.01510903 ... -0.06985617 -0.02305336
   0.05401625]
 [-0.01334469  0.01704533 -0.05602343 ...  0.01448949 -0.04619199
  -0.05617152]
 [ 0.05268429 -0.01391449  0.04169049 ...  0.04957248 -0.00793167
   0.04924917]
 ...
 [ 0.07666269  0.00141271  0.03278234 ...  0.00271717 -0.03690447
  -0.04324872]
 [ 0.07673816  0.00217168 -0.06574667 ...  0.01257792  0.0316073
   0.0201044 ]
 [ 0.04707664 -0.00459063 -0.02711707 ... -0.00465905 -0.00946341
   0.02349724]], shape=(3263, 512), dtype=float32)


In [42]:
print(y_train)

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64


In [41]:
# features_to_drop = ['id', 'keyword','location','text','location_clean','text_clean', 'hashtags', 'mentions','links']

# X_train = train.drop(columns = features_to_drop + ['target'])
# X_test = test.drop(columns = features_to_drop)


params = {
    'learning_rate': 0.06,
    'n_estimators': 1500,
    'colsample_bytree': 0.5
}

clf = LGBMClassifier(**params)

clf.fit(X_train['outputs'], y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.06, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [44]:
y_test = clf.predict(X_test['outputs'])

submit = sub_sample.copy()
submit.target = y_test
submit.to_csv('submit_lightgbm.csv',index=False)

In [47]:
predictions = clf.predict(X_train['outputs'])
# F-1 score
print ("Training set f1_score :", np.round(f1_score(y_train, predictions),5))

Training set f1_score : 0.99602


In [50]:
# Matriz de Confucion
pd.DataFrame(confusion_matrix(y_train, clf.predict(X_train['outputs'])))

,0,1
0,4333,9
1,17,3254


## Evaluacion del Modelo

In [51]:
# Cross validation
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=123)
cv_score = cross_val_score(clf, X_train['outputs'], y_train, cv=cv, scoring='f1')
print('Cross validation F-1 score: %.3f' %np.mean(cv_score))

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([6445, 3870, 2431, ..., 3454, 7533, 3582])